# Branch & Bound per Knapsack

In [99]:
import numpy as np

## Istanza di Knapsack

In [100]:
N = 20
P = np.random.randint(low=1, high=40, size=N)
print(f"P: {P}")
W = np.random.randint(low=1, high=40, size=N)
print(f"W: {W}")
C = 50

def order_by_ratio(P, W):
    indexes = np.argsort(P/W)[::-1]
    print(f"indexes: {indexes}")
    new_p = np.array([P[i] for i in indexes])
    new_w = np.array([W[i] for i in indexes])
    return new_p, new_w

P, W = order_by_ratio(P, W)
print(f"Ordered P: {P}")
print(f"Ordered W: {W}")

P: [34 15 34  3 34 29 38 32 16 23 39 36 31 20 19 34  7 23 39 34]
W: [28 12 27  4 38 39 18 25 32  8 18 12 34 19  2 36  1  8 16 39]
indexes: [14 16 11  9 17 18 10  6  7  2  1  0 13 15 12  4 19  3  5  8]
Ordered P: [19  7 36 23 23 39 39 38 32 34 15 34 20 34 31 34 34  3 29 16]
Ordered W: [ 2  1 12  8  8 16 18 18 25 27 12 28 19 36 34 38 39  4 39 32]


## Implementazione State Space Tree

In [101]:
class Node:
    def __init__(self, label):
        self.label = label
        self.children = []
        self.parent = None

    def setParent(self, parent):
        self.parent = parent

    def addChildren(self, children):
        self.children.extend(children)

    def __str__(self):
        return str(self.label)

class StateSpaceTree:
    def __init__(self, max_length, values, repetitions=False):
        self.root = Node([])
        self.nodes = [self.root]
        self.max_length = max_length
        self.values = values
        self.repetions = repetitions

    def expand(self, node):
        if len(node.label) == self.max_length:
            return []
        children = []
        for value in self.values:
            if self.repetions and value in node.label:
                continue
            label = node.label + [value]
            child = Node(label)
            child.setParent = node
            children.append(child)
        node.addChildren(children)
        self.nodes.extend(children)
        return children

## Implementazione Lista dei Live Node

### Definizione funzioni guadagno

In [102]:
def fw(node):
    x = np.array(node.label)
    return np.sum(x*W[0:len(x)])

def fp(node):
    x = np.array(node.label)
    return np.sum(x*P[0:len(x)])

def g(node):
    x = np.array(node.label)
    i = len(x)
    if i == N:
        return 0
    res = 0
    tot_weight = np.sum(x*W[0:i])
    while tot_weight + W[i] <= C:
        res += P[i]
        tot_weight += W[i]
        i += 1
    res += (C - tot_weight) / W[i] * P[i]
    return res

### Definizione classe Lista

In [103]:
class LiveNodesList:
    def __init__(self, root, strategy):
        self.live_nodes = [root]
        self.strategy = strategy

    def select(self):
        if self.strategy == "LIFO":
            last = self.live_nodes[-1]
            self.live_nodes = self.live_nodes[:-1]
            return last
        if self.strategy == "FIFO":
            first = self.live_nodes[0]
            self.live_nodes = self.live_nodes[1:]
            return first
        if self.strategy == "LC":
            max_node = self.live_nodes[0]
            max_profit = fp(max_node) + g(max_node)
            for node in self.live_nodes[1:]:
                profit = fp(node) + g(node)
                if profit > max_profit:
                    max_profit = profit
                    max_node = node
            self.live_nodes.remove(max_node)
            return max_node
                
    def insert(self, node):
        self.live_nodes.append(node)

    def empty(self):
        return len(self.live_nodes) == 0
    
    def set_strategy(self, strategy):
        self.strategy = strategy

    def __str__(self):
        return str([node.label for node in self.live_nodes])

## Algoritmo di visita

In [104]:
def initializeMaxSolution():
    z = 0
    x = [0]*N
    tot_weight = 0
    i = 0
    while tot_weight + W[i] <= C:
        z += P[i]
        tot_weight += W[i]
        x[i] = 1
        i += 1
    return z, x

def branchAndBoundForKP(tree, strategy="LC"):
    z_max, x_max = initializeMaxSolution()
    print(f"Initial greedy guess is: {x_max}, profit: {z_max}, total weight: {np.sum(x_max*W)}")
    live_nodes = LiveNodesList(tree.root, strategy=strategy)
    iteration = 0
    while not live_nodes.empty():
        print(f"\nIteration {iteration}")
        print(f"Live nodes: {live_nodes}")
        e_node = live_nodes.select()
        print(f"E-node: {e_node}")
        if not (fw(e_node) > C or (fp(e_node) + g(e_node)) <= z_max):
            z = fp(e_node)
            if z >= z_max:
                z_max = z
                x_max = e_node.label
                print(f"A new max has been found: {z_max}")
            print("The E-node will be expanded")
            new_nodes = tree.expand(e_node)
            for node in new_nodes:
                live_nodes.insert(node)
        else:
            print("The E-node will not be expanded.")
        iteration += 1
    x_max += [0]*(N-len(x_max))
    return x_max, z_max

tree = StateSpaceTree(N, [0,1])
x_star, z_star = branchAndBoundForKP(tree)
print(f"\nThe answer is: {x_star}, profit: {z_star}, total weight: {np.sum(x_star*W)}")

Initial greedy guess is: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], profit: 147, total weight: 47

Iteration 0
Live nodes: [[]]
E-node: []
The E-node will be expanded

Iteration 1
Live nodes: [[0], [1]]
E-node: [1]
The E-node will be expanded

Iteration 2
Live nodes: [[0], [1, 0], [1, 1]]
E-node: [1, 1]
The E-node will be expanded

Iteration 3
Live nodes: [[0], [1, 0], [1, 1, 0], [1, 1, 1]]
E-node: [1, 1, 1]
The E-node will be expanded

Iteration 4
Live nodes: [[0], [1, 0], [1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1]]
E-node: [1, 1, 1, 1]
The E-node will be expanded

Iteration 5
Live nodes: [[0], [1, 0], [1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]
E-node: [1, 1, 1, 1, 1]
The E-node will be expanded

Iteration 6
Live nodes: [[0], [1, 0], [1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1]]
E-node: [1, 1, 1, 1, 1, 1]
A new max has been found: 147
The E-node will be expanded

Iteration 7
Live nodes: [[0], [1, 0], [1, 1, 0], [1, 1, 1